# Biodiversity Data Journal 크롤러

- 2013 ~ 2019년 (vol.1 ~ vol.7) 게재 논문 저자와 이메일을 크롤링하는 크롤러입니다.
- 크롤링 결과는 'Platform', 'Year', 'Volume', 'Author', 'Email', 'URL'의 6개 칼럼으로 구성된 데이터프레임으로 정리하였습니다.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [130]:
df = pd.DataFrame(columns = ['Platform', 'Year', 'Volume', 'Author', 'Email', 'URL'])

driver = webdriver.Chrome('../chromedriver')

root = 'https://bdj.pensoft.net/browse_journal_articles.php?journal_name=bdj&form_name=filter_articles&sortby=0&journal_id=1&search_in_=0&to_date=31%2F12%2F2019&section_type[]=8&p='

wait = WebDriverWait(driver, 10)

for k in range(10):
    board_url = root + str(k)
    driver.get(board_url)
    articles = driver.find_elements_by_class_name('col_no_padding.col-xs-11.start-xs.articleHeadline')
    article_url = [article.find_element_by_tag_name('a').get_attribute('href') for article in articles]
    for url in article_url:
        driver.get(url)
        article_id = driver.find_element_by_id('article-id')
        volume = article_id.text.split()[3].strip(':')
        year = article_id.text.split('(')[-1].strip(')')[-4:]
        #iframe 변경
        iframe = driver.find_element_by_xpath('//*[@id="articleIframe"]')
        driver.switch_to.frame(iframe)
        #드롭다운 버튼 클릭
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'P-Article-Preview-Names.unselectable')))
        button = driver.find_element_by_class_name('P-Article-Preview-Names.unselectable').find_element_by_tag_name('a')
        button.click()
        auth_info = driver.find_element_by_class_name('P-Article-Info-Block-Row').text
        try:   #교신 저자 2명 이상인 경우
            authors = auth_info.lstrip('Corresponding author: ').split(',')
            for author in authors:
                author_name = author.split('(')[0].strip()
                author_email = author.split('(')[1].strip(')')
                df = df.append({'Platform': 'BDJ', 'Year': year, 'Volume': volume, 'Author': author_name, 'Email': author_email, 'URL': url}, ignore_index = True)
        except:   #교신 저자 1명인 경우
            author = auth_info.split('(')[0].lstrip('Corresponding author: ').rstrip()
            email = auth_info.split('(')[1].strip(')')
            df = df.append({'Platform': 'BDJ', 'Year': year, 'Volume': volume, 'Author': author, 'Email': email, 'URL': url}, ignore_index = True)
print('finished')

finshed


In [129]:
df

,Platform,Year,Volume,Author,Email,URL
0,BDJ,2019,7,Frederik Van de Perre,frederik.vandeperre@uantwerpen.be,https://bdj.pensoft.net/article/46948/
1,BDJ,2019,7,Bang Van Tran,vn.vanbang@gmail.com,https://bdj.pensoft.net/article/39233/
2,BDJ,2019,7,Reinaldo Imbrozio Barbosa,reinaldo@inpa.gov.br,https://bdj.pensoft.net/article/47025/
3,BDJ,2019,7,Rui B. Elias,rui.mp.elias@uac.pt,https://bdj.pensoft.net/article/38687/
4,BDJ,2019,7,Stefano Mammola,stefano.mammola@helsinki.fi,https://bdj.pensoft.net/article/38492/
...,...,...,...,...,...,...
136,BDJ,2014,2,Julissa Roncal,jroncal@mun.ca,https://bdj.pensoft.net/article/1078/
137,BDJ,2014,2,Gary D Powney,gary.powney@ceh.ac.uk,https://bdj.pensoft.net/article/1041/
138,BDJ,2014,2,Bonaventure Sonké,bsonke_1999@yahoo.com,https://bdj.pensoft.net/article/1049/
139,BDJ,2014,2,Sarah Faulwetter,sarifa@hcmr.gr,https://bdj.pensoft.net/article/1024/


In [131]:
df.to_csv('../csv/BDJ.txt', encoding = 'utf-8')